In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
scSpark = SparkSession \
        .builder \
        .appName("memfilter date") \
        .config("spark.jars", "postgresql-42.3.2.jar") \
        .getOrCreate()

22/07/25 15:46:48 WARN Utils: Your hostname, rifatqs-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/07/25 15:46:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/25 15:46:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/25 15:46:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_invoice = scSpark.read \
            .format("jdbc") \
            .option('url', 'jdbc:postgresql://localhost/small_project1') \
            .option('dbtable', 'invoice') \
            .option('user', 'postgres') \
            .option('password', 'postgres') \
            .option('driver', 'org.postgresql.Driver') \
            .load()

In [4]:
df_invoice.show(5)

+----+--------------+------+------+-----+--------------------+-------------------+
|item|     invoiceid|userid|amount|notes|       paymentstatus|               date|
+----+--------------+------+------+-----+--------------------+-------------------+
|   1|INVCLSYLFY0001|     2| 65000| NULL|Successfully capt...|2017-03-30 09:21:53|
|   1|INVCLSYLFY0002|     2| 65000| NULL|Waiting customer ...|2017-03-30 09:44:38|
|   2|INVCLSYLFY0003|     5| 65000| NULL|Waiting customer ...|2017-03-30 10:01:35|
|   1|INVCLSYLFY0004|     2| 65000| NULL|Successfully capt...|2017-03-30 10:39:39|
|   1|INVCLSYLFY0005|     8| 65000| NULL|Successfully capt...|2017-03-30 10:50:19|
+----+--------------+------+------+-----+--------------------+-------------------+
only showing top 5 rows



In [5]:
df_invoice.groupBy('paymentstatus').count().show()

+--------------------+-----+
|       paymentstatus|count|
+--------------------+-----+
|Payment using ech...|  230|
|Waiting customer ...|    1|
|Waiting customer ...|  287|
|Transaction succe...| 1799|
|Successfully capt...|  104|
+--------------------+-----+



In [10]:
df_sudahbayar1 = df_invoice.filter(df_invoice.paymentstatus == 'Successfully captured using credit_card')

In [11]:
df_sudahbayar1.count()

104

In [12]:
df_sudahbayar2 = df_invoice.filter(df_invoice.paymentstatus == 'Transaction successfully transfered using credit_card')

In [13]:
df_sudahbayar2.count()

1799

In [14]:
df_sudahbayar_final = df_sudahbayar1.union(df_sudahbayar2)

In [15]:
df_sudahbayar_final.count()

1903

In [16]:
df_sudahbayar_final.show(10)

+----+--------------+------+------+-----+--------------------+-------------------+
|item|     invoiceid|userid|amount|notes|       paymentstatus|               date|
+----+--------------+------+------+-----+--------------------+-------------------+
|   1|INVCLSYLFY0001|     2| 65000| NULL|Successfully capt...|2017-03-30 09:21:53|
|   1|INVCLSYLFY0004|     2| 65000| NULL|Successfully capt...|2017-03-30 10:39:39|
|   1|INVCLSYLFY0005|     8| 65000| NULL|Successfully capt...|2017-03-30 10:50:19|
|   1|INVCLSYLFY0007|    10| 65000| NULL|Successfully capt...|2017-03-30 11:19:16|
|   1|INVCLSYLFY0008|    10| 65000| NULL|Successfully capt...|2017-03-30 11:20:34|
|   1|INVCLSYLFY0009|    10| 65000| NULL|Successfully capt...|2017-03-30 11:22:40|
|   1|INVCLSYLFY0010|    22| 65000| NULL|Successfully capt...|2017-04-07 23:19:35|
|   1|INVCLSYLFY0012|    24| 65000| NULL|Successfully capt...|2017-04-08 01:05:10|
|   1|INVCLSYLFY0013|    25| 65000| NULL|Successfully capt...|2017-04-08 01:08:04|
|   

In [17]:
df_sudahbayar_awal2017 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2017-01-01') & (df_sudahbayar_final.date < '2017-4-01'))

In [19]:
df_sudahbayar_awal2018 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2018-01-01') & (df_sudahbayar_final.date < '2018-04-01'))

In [32]:
df_sudahbayar_awal2019 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2019-01-01') & (df_sudahbayar_final.date < '2019-04-01'))

In [34]:
df_sudahbayar_awal2020 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2020-01-01') & (df_sudahbayar_final.date < '2020-04-01'))

In [36]:
df_sudahbayar_awaltahun1 = df_sudahbayar_awal2017.union(df_sudahbayar_awal2018)

In [37]:
df_sudahbayar_awaltahun2 = df_sudahbayar_awal2019.union(df_sudahbayar_awal2020)

In [38]:
df_sudahbayar_awaltahun_final = df_sudahbayar_awaltahun1.union(df_sudahbayar_awaltahun2)

In [39]:
df_sudahbayar_awaltahun_final.count()

175

In [42]:
df_sudahbayar_awaltahun_final.show()

+----+--------------+------+------+-----+--------------------+-------------------+
|item|     invoiceid|userid|amount|notes|       paymentstatus|               date|
+----+--------------+------+------+-----+--------------------+-------------------+
|   1|INVCLSYLFY0001|     2| 65000| NULL|Successfully capt...|2017-03-30 09:21:53|
|   1|INVCLSYLFY0004|     2| 65000| NULL|Successfully capt...|2017-03-30 10:39:39|
|   1|INVCLSYLFY0005|     8| 65000| NULL|Successfully capt...|2017-03-30 10:50:19|
|   1|INVCLSYLFY0007|    10| 65000| NULL|Successfully capt...|2017-03-30 11:19:16|
|   1|INVCLSYLFY0008|    10| 65000| NULL|Successfully capt...|2017-03-30 11:20:34|
|   1|INVCLSYLFY0009|    10| 65000| NULL|Successfully capt...|2017-03-30 11:22:40|
|   1|INVCLSYLFY1156|   146| 65000| NULL|Transaction succe...|2018-01-03 10:02:33|
|   1|INVCLSYLFY1157|  2640| 65000| NULL|Transaction succe...|2018-01-09 09:54:03|
|   1|INVCLSYLFY1158|   146| 65000| NULL|Transaction succe...|2018-02-05 21:16:09|
|   

In [43]:
df_sudahbayar_awaltahun_final.write \
    .format("jdbc") \
    .option('url', 'jdbc:postgresql://localhost/small_project1') \
    .option('dbtable', 'awal_tahun') \
    .option('user', 'postgres') \
    .option('password', 'postgres') \
    .option('driver','org.postgresql.Driver') \
    .mode('append') \
    .save()

In [49]:
df_sudahbayar_akhir2017 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2017-10-01') & (df_sudahbayar_final.date <= '2017-12-31'))

In [50]:
df_sudahbayar_akhir2018 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2018-10-01') & (df_sudahbayar_final.date <= '2018-12-31'))

In [51]:
df_sudahbayar_akhir2019 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2019-10-01') & (df_sudahbayar_final.date <= '2019-12-31'))

In [52]:
df_sudahbayar_akhir2020 = df_sudahbayar_final.filter (( df_sudahbayar_final.date >= '2020-10-01') & (df_sudahbayar_final.date <= '2020-12-31'))

In [53]:
df_sudahbayar_akhirtahun1 = df_sudahbayar_akhir2017.union(df_sudahbayar_akhir2018)

In [54]:
df_sudahbayar_akhirtahun2 = df_sudahbayar_akhir2019.union(df_sudahbayar_akhir2020)

In [55]:
df_sudahbayar_akhirtahun_final = df_sudahbayar_akhirtahun1.union(df_sudahbayar_akhirtahun2)

In [56]:
df_sudahbayar_akhirtahun_final.count()

761

In [57]:
df_sudahbayar_akhirtahun_final.show()

+----+--------------+------+------+-----+--------------------+-------------------+
|item|     invoiceid|userid|amount|notes|       paymentstatus|               date|
+----+--------------+------+------+-----+--------------------+-------------------+
|   1|INVCLSYLFY0442|   218| 65000| NULL|Successfully capt...|2017-10-02 19:38:40|
|   1|INVCLSYLFY0456|   905| 65000| NULL|Successfully capt...|2017-10-03 17:14:53|
|   1|INVCLSYLFY0478|   927| 65000| NULL|Successfully capt...|2017-10-03 17:50:13|
|   1|INVCLSYLFY0479|   928| 65000| NULL|Successfully capt...|2017-10-03 17:51:12|
|   1|INVCLSYLFY0480|   929| 65000| NULL|Successfully capt...|2017-10-03 17:51:53|
|   1|INVCLSYLFY0481|   930| 65000| NULL|Successfully capt...|2017-10-03 17:57:30|
|   1|INVCLSYLFY0508|   957| 65000| NULL|Successfully capt...|2017-10-03 18:30:59|
|   1|INVCLSYLFY0509|   958| 65000| NULL|Successfully capt...|2017-10-03 18:32:25|
|   1|INVCLSYLFY0510|   959| 65000| NULL|Successfully capt...|2017-10-03 18:34:35|
|   

In [58]:
df_sudahbayar_akhirtahun_final.write \
    .format("jdbc") \
    .option('url', 'jdbc:postgresql://localhost/small_project1') \
    .option('dbtable', 'akhir_tahun') \
    .option('user', 'postgres') \
    .option('password', 'postgres') \
    .option('driver','org.postgresql.Driver') \
    .mode('append') \
    .save()